In [1]:
import torch
from torch import nn
from peft import LoraConfig,get_peft_model,PeftModel

# 1. 自定义模型适配

In [2]:
net1 = nn.Sequential(
    nn.Linear(10,10),
    nn.ReLU(),
    nn.Linear(10,2)
)
net1

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [3]:
for name,parm in net1.named_parameters():
    print(name)
    

0.weight
0.bias
2.weight
2.bias


In [4]:
config = LoraConfig(target_modules=["0"])


In [5]:
model1 = get_peft_model(net1,config)

In [6]:
model1

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

# 2. 多适配器加载与切换

In [7]:
net2 = nn.Sequential(
    nn.Linear(10,10),
    nn.ReLU(),
    nn.Linear(10,2)
)
net2

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [8]:
config1 = LoraConfig(target_modules=["0"])
model2 = get_peft_model(net2,config1)
model2.save_pretrained("./loraA")

In [9]:
config2 = LoraConfig(target_modules=["2"])
model2 = get_peft_model(net2, config2)
model2.save_pretrained("./loraB")

/Users/liuzhen/Documents/PEFT_Study/.venv/lib/python3.12/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/Users/liuzhen/Documents/PEFT_Study/.venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [10]:
model2 = PeftModel.from_pretrained(net2, model_id="./loraA/", adapter_name="loraA")
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=2, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
    

In [11]:
model2.load_adapter("./loraB/", adapter_name="loraB")
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=2, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
          (loraB): Identity()
        )
    

In [12]:
model2.active_adapter

'loraA'

In [13]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[ 0.4554, -2.5740]])

In [14]:
for name, param in model2.named_parameters():
    print(name, param)

base_model.model.0.base_layer.weight Parameter containing:
tensor([[-0.0621,  0.0434,  0.0322, -0.1129,  0.2890, -0.0049,  0.2818,  0.0177,
          0.2881, -0.0446],
        [ 0.0597, -0.2148, -0.2554,  0.0022, -0.1270, -0.0511, -0.2159,  0.0690,
          0.0423, -0.2214],
        [ 0.2724, -0.2535, -0.2873, -0.2022,  0.1348, -0.0765,  0.0122,  0.0028,
          0.1878,  0.2557],
        [ 0.0083, -0.2733, -0.0428,  0.0069, -0.1523, -0.0485, -0.0322, -0.1476,
          0.1904, -0.2775],
        [ 0.0892,  0.1389,  0.1269, -0.2619,  0.1822, -0.1095, -0.1189, -0.2207,
         -0.0865,  0.1196],
        [-0.0677,  0.2571, -0.3158, -0.0199, -0.1225, -0.2901,  0.0451,  0.1588,
          0.1946, -0.3010],
        [-0.1767,  0.0783,  0.1378,  0.1358, -0.2111,  0.2290,  0.1890,  0.0943,
          0.2156,  0.1094],
        [ 0.1753,  0.2050, -0.2076,  0.1502,  0.1677,  0.1737, -0.0322,  0.2930,
          0.1243,  0.1071],
        [ 0.2326, -0.2203,  0.3072,  0.2820, -0.2275, -0.2437, -0.178

In [15]:
for name, param in model2.named_parameters():
    if name in ["base_model.model.0.lora_A.loraA.weight", "base_model.model.0.lora_B.loraA.weight"]:
        param.data = torch.ones_like(param)

In [16]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[ 167.4115, -182.9486]])

In [17]:
model2.set_adapter("loraB")

In [18]:
model2.active_adapter

'loraB'

In [19]:
model2(torch.arange(0,10).view(1,10).float())

tensor([[ 0.4554, -2.5740]], grad_fn=<AddBackward0>)

# 禁用适配器

In [20]:
model2.set_adapter("loraA")

In [21]:
model2(torch.arange(0,10).view(1,10).float())

tensor([[ 167.4115, -182.9486]], grad_fn=<AddmmBackward0>)

In [22]:
with model2.disable_adapter():
    print(model2(torch.arange(0,10).view(1,10).float()))

tensor([[ 0.4554, -2.5740]])


In [33]:
# 永久禁用：手动关闭所有适配器层
model2.base_model.disable_adapter_layers()

# 测试：此时应该只有 Base Model 的效果
print("禁用后输出：")
print(model2(torch.arange(0, 10).view(1, 10).float()))

禁用后输出：
tensor([[ 0.4554, -2.5740]])


In [34]:
model2.active_adapter

'default'

In [35]:
# 1. 重新开启适配器层
model2.base_model.enable_adapter_layers()

# 2. 确保指定了你要用的适配器（以防万一）
model2.set_adapter("loraA") 

# 测试：此时恢复 LoRA 效果
print("恢复后输出：")
print(model2(torch.arange(0, 10).view(1, 10).float()))

恢复后输出：
tensor([[ 167.4115, -182.9486]], grad_fn=<AddmmBackward0>)
